## Gun Violence Project
The goal of this project is to look at gun violence incidents in the United States from 1991-2023 and draw correlations from the gun legislation that was in place in each state at the time of the event. The hope is to find some correlation that will show that certain legislation is effective at curbing gun violence.

##### Setup
* We are going to use several different packages to ease in ingest,cleaning, transformation, and visualization of the data. These packages will be imported in the cell below

In [1]:
from pathlib import Path
import pandas as pd
import numpy as np
import glob


* First we are going to read all of the files for gun violence incidents from the folder and concatenate them into one dataframe. For this we are using pandas and glob. Glob will allow us to search through the folder holding our files using aa pattern and then collect the names into a list.

In [2]:
#Reading in Gun Violence Incident Data First
path = '../Gun Violence Project/Gun Violence Archive Data/'
csv_files = glob.glob(path + "/*.csv")

df_list = (pd.read_csv(file) for file in csv_files)

GV_Master_Raw   = pd.concat(df_list, ignore_index=True)

#Reading in Gun Legislation Data
GL_Master_Raw=pd.read_csv("../Gun Violence Project/Gun Laws/gun_laws.csv")



#### Cleaning and Transformation

* Cleaning GV_Master_Raw

In [3]:
#Now we need to extract the year from the dates. The first line will convert Incident Date to a datetime type
#and the second line will extract the year.
GV_Master_Raw['Incident Date'] = pd.to_datetime(GV_Master_Raw['Incident Date'])
GV_Master_Raw['Incident Date'] = GV_Master_Raw['Incident Date'].dt.strftime('%Y')
GV_Master_Raw.head(5)

,Incident ID,Incident Date,State,City Or County,Address,# Victims Injured,# Victims Killed,# Subjects-Suspects Injured,# Subjects-Suspects Killed,# Subjects-Suspects Arrested,Operations
0,496703,2013,Ohio,Barberton,300 block of Seventh Street Northeast,1,3,0,0,1,NaN
1,496684,2013,New York,Brooklyn,60 Glenmore Ave,6,0,0,0,0,NaN
2,496668,2013,Alabama,Montgomery,954 Highland Ave,5,3,0,0,2,NaN
3,496634,2013,Louisiana,Slidell,2144 First St,6,2,0,0,1,NaN
4,496584,2013,Louisiana,Lockport,313 Tenth St,3,2,0,1,0,NaN


In [4]:
# In this cell we are using len to get a count of how many rows we have in each DF.
print(len(GL_Master_Raw.index))
print(len(GV_Master_Raw.index))

1450
6346


In [5]:

# Dropping the extra columns in the GV DF that won't be needed.
GV_Clean = GV_Master_Raw[['Incident ID', 'Incident Date','State','# Victims Injured','# Victims Killed']]

#Removing duplicates in the GV DF based on Incident ID. There are duplicates because 
# some of the csv files have overlapping data. Printing Len again to confirm that duplicates were dropped.
GV_Clean=GV_Clean.drop_duplicates(subset=['Incident ID'],keep='first')


#Renaming Incident Date to Year for merging operations.
GV_Clean.rename(columns = {'Incident Date':'Year'}, inplace = True)

GVM_row=len(GV_Master_Raw.index)
GVC_row=len(GV_Clean.index)
print(GVM_row-GVC_row, "Duplicate rows were removed")

1988 Duplicate rows were removed


* Cleaning GL_Master_Raw

In [6]:
# Renaming columns in GL_MASTER so that they are the same as GV_CLEAN for merging.

GL_Master_Raw.rename(columns = {'STATE':'State','YEAR':'Year'}, inplace = True)
GL_Clean=GL_Master_Raw

GL_Clean['Year'] = pd.to_datetime(GL_Clean['Year'])
GL_Clean['Year'] = GL_Clean['Year'].dt.strftime('%Y')
GL_Clean.head()

,State,Year,FELONY,INVCOMMITMENT,INVOUTPATIENT,DANGER,DRUGMISDEMEANOR,ALCTREATMENT,ALCOHOLISM,RELINQUISHMENT,...,EXPARTEDATING,DVROSURRENDER,DVROSURRENDERNOCONDITIONS,DVROSURRENDERDATING,EXPARTESURRENDER,EXPARTESURRENDERNOCONDITIONS,EXPARTESURRENDERDATING,DVROREMOVAL,STALKING,LAWTOTAL
0,Alabama,1970,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,15
1,Alaska,1970,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,10
2,Arizona,1970,0,1,1,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,12
3,Arkansas,1970,1,1,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,15
4,California,1970,1,1,0,1,0,0,1,0,...,0,0,0,0,0,0,0,0,0,58


In [7]:
#Before we attempt to merge the two DFs we need to ensure that there are no NA values in Year or State

print("There are",(GV_Clean['Year'].isna().sum()),"NA values in the Year column of GV_Clean")
print("There are",(GV_Clean['State'].isna().sum()),"NA values in the State column of GV_Clean")
print("There are",(GL_Clean['Year'].isna().sum()),"NA values in the Year column of GL_Clean")
print("There are",(GL_Clean['State'].isna().sum()),"NA values in the State column of GV_Clean")

print(GV_Clean['Year'].dtypes)
print(GL_Clean['Year'].dtypes)

There are 0 NA values in the Year column of GV_Clean
There are 0 NA values in the State column of GV_Clean
There are 0 NA values in the Year column of GL_Clean
There are 0 NA values in the State column of GV_Clean
object
object


In [11]:
#Here we are ensuring that the columns we will use in our merge function below are of the same type 
#and don't contain white spaces.

GV_Clean['Year'] = GV_Clean['Year'].astype('int')
GL_Clean['Year'] = GL_Clean['Year'].astype('int')
GV_Clean['State'] = GV_Clean['State'].astype(str)
GL_Clean['State'] = GL_Clean['State'].astype(str)
GV_Clean['State'] = GV_Clean['State'].str.strip()
GL_Clean['State'] = GL_Clean['State'].str.strip()
GV_Clean.head(10)

,Incident ID,Year,State,# Victims Injured,# Victims Killed
0,496703,2013,Ohio,1,3
1,496684,2013,New York,6,0
2,496668,2013,Alabama,5,3
3,496634,2013,Louisiana,6,2
4,496584,2013,Louisiana,3,2
5,495577,2013,New York,3,1
6,495588,2013,New Jersey,2,3
7,495566,2013,Michigan,3,1
8,495557,2013,North Carolina,4,0
9,495548,2013,New Jersey,4,1


In [13]:
#Once again checking for duplicate rows in the data sets. 
duplicates_GV = GV_Clean.duplicated(subset=['State', 'Year','Incident ID'])
duplicates_GL = GL_Clean.duplicated(subset=['State', 'Year'])
duplicates_GV.head(5)

0    False
1    False
2    False
3    False
4    False
dtype: bool

In [14]:
#Currently this merge operation is having an issue 
#because we are merging on Year and State and there are duplicate records for those two columns, 
#but it's because there were multiple GV events in some states during the same year.
GVP_Clean = pd.merge(GV_Clean, GL_Clean, on=['Year','State'],how='left')

In [15]:


GVP_Clean = GVP_Clean[(GVP_Clean.Year >= 2013) & (GVP_Clean.Year <= 2019)]

GVP_Clean.head(200)

,Incident ID,Year,State,# Victims Injured,# Victims Killed,FELONY,INVCOMMITMENT,INVOUTPATIENT,DANGER,DRUGMISDEMEANOR,...,EXPARTEDATING,DVROSURRENDER,DVROSURRENDERNOCONDITIONS,DVROSURRENDERDATING,EXPARTESURRENDER,EXPARTESURRENDERNOCONDITIONS,EXPARTESURRENDERDATING,DVROREMOVAL,STALKING,LAWTOTAL
0,496703,2013,Ohio,1,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,496684,2013,New York,6,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,496668,2013,Alabama,5,3,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,496634,2013,Louisiana,6,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,496584,2013,Louisiana,3,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,486005,2013,New Jersey,5,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
196,485979,2013,Tennessee,2,2,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
197,485885,2013,Pennsylvania,3,1,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
198,485874,2013,North Carolina,5,0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
